In [1]:
import formulas
import storage
import pandas as pd
import os

if __name__ == '__main__':

    website_creator_path = os.getcwd()
    formula_file_path = website_creator_path + os.path.sep + 'formulas.csv'
    order_file_dir = website_creator_path + os.path.sep + 'order_files'
    docs_dir = storage.get_docs_path(website_creator_path)
    
    # storage.delete_directory_if_it_exists(docs_dir)

    
    df_formulas =  storage.get_formulas(formula_file_path)
    df_sort_orders = storage.get_sort_orders(order_file_dir)    

    # formula_paths = df_formulas[['State', 'Subject code']].drop_duplicates().to_numpy(dtype = str)
    # formulas.create_formula_directories(docs_dir, formula_paths)
    # formulas.create_formula_index_files(docs_dir, df_sort_orders)
    # formulas.create_formula_files(docs_dir, df_formulas)

    # df_calculus = formulas.df_calculus_summary(df_formulas)
    # styler_calculus = formulas.styler_calculus_summary(df_calculus, formulas.formulas_on_formula_sheet(df_formulas))
    # display(styler_calculus)

In [3]:
docs_dir

'/home/charl/Onedrive/Documents_Charl/Computer_Technical/Programming_GitHub/AustralianSchoolMaths/content/docs'

In [11]:
df_formulas[['State', 'Category']]

,State,Category
0,NSW,Indices
1,NSW,Indices
2,NSW,Indices
3,NSW,Indices
4,NSW,Indices
...,...,...
59,NSW,Statistics
60,NSW,Statistics
61,NSW,Statistics
62,NSW,Surds


In [18]:
[website_creator_path + os.path.sep + x for x in os.listdir(website_creator_path + os.path.sep + 'order_files')]

['/home/charl/Onedrive/Documents_Charl/Computer_Technical/Programming_GitHub/AustralianSchoolMaths/WebsiteCreator/order_states.csv',
 '/home/charl/Onedrive/Documents_Charl/Computer_Technical/Programming_GitHub/AustralianSchoolMaths/WebsiteCreator/order_subjects.csv',
 '/home/charl/Onedrive/Documents_Charl/Computer_Technical/Programming_GitHub/AustralianSchoolMaths/WebsiteCreator/Test.txt']

In [29]:
for item in os.listdir(website_creator_path + os.path.sep + 'order_files'):
    file_ext = os.path.splitext(item)[1]
    if file_ext.upper() == '.CSV':
        print(website_creator_path + os.path.sep + item)

/home/charl/Onedrive/Documents_Charl/Computer_Technical/Programming_GitHub/AustralianSchoolMaths/WebsiteCreator/order_states.csv
/home/charl/Onedrive/Documents_Charl/Computer_Technical/Programming_GitHub/AustralianSchoolMaths/WebsiteCreator/order_subjects.csv


In [3]:
import storage
order_folder = website_creator_path + os.path.sep + 'order_files'
display(storage.csv_files_in_dir(order_folder))

['/home/charl/Onedrive/Documents_Charl/Computer_Technical/Programming_GitHub/AustralianSchoolMaths/WebsiteCreator/order_files/order_states.csv',
 '/home/charl/Onedrive/Documents_Charl/Computer_Technical/Programming_GitHub/AustralianSchoolMaths/WebsiteCreator/order_files/order_subjects.csv']